In [4]:
!pip install pandas==1.5.3
!pip install numpy
!pip install datasets==2.14.5
!pip install torch
!pip install transformers
!pip install scikit-learn
!pip install accelerate
!pip install evaluate
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 69.2 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.2
    Uninstalling pandas-2.0.2:
      Successfully uninstalled pandas-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which 

In [5]:
import pandas as pd
import numpy as np
import datasets
import torch
import os
from datasets import load_dataset, load_from_disk, concatenate_datasets
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer
from sklearn.model_selection import train_test_split
import evaluate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
pd.__version__

'1.5.3'

In [7]:
datasets.__version__

'2.14.5'

In [8]:
model_name = 'Helsinki-NLP/opus-mt-de-de'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
dataset = load_dataset('/kaggle/input/eitasp', data_files={'train': 'train.csv', 'test': 'test.csv', 'validation': 'validation.csv'})
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'de', 'ch_dialect'],
        num_rows: 24967
    })
    test: Dataset({
        features: ['Unnamed: 0', 'de', 'ch_dialect'],
        num_rows: 4639
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'de', 'ch_dialect'],
        num_rows: 1315
    })
})

In [10]:
def process_data(batch):
    # Tokenize source and target sequences separately
    source_encodings = tokenizer(batch['ch_dialect'], truncation=True, padding='max_length', max_length=512, return_tensors="pt")
    target_encodings = tokenizer(batch['de'], truncation=True, padding='max_length', max_length=512, return_tensors="pt")

    # For training, the decoder_input_ids are usually the same as the target input_ids (or shifted, as described before)
    decoder_input_ids = target_encodings["input_ids"]

    return {
        "input_ids": source_encodings["input_ids"],
        "attention_mask": source_encodings["attention_mask"],
        "decoder_input_ids": decoder_input_ids,
        "labels": decoder_input_ids
    }

tokenized_datasets = dataset.map(process_data, batched=True)

Map:   0%|          | 0/24967 [00:00<?, ? examples/s]

Map:   0%|          | 0/4639 [00:00<?, ? examples/s]

Map:   0%|          | 0/1315 [00:00<?, ? examples/s]

In [11]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
# Define training arguments and trainer
training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    save_steps=500,
    output_dir="./results",
    overwrite_output_dir=True,
    remove_unused_columns=True,
    save_total_limit=3
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Fine-tune
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
500,0.113200,0.042482
1000,0.038500,0.027039
1500,0.027500,0.021013
2000,0.022300,0.018568


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=2343, training_loss=0.04599613803500453, metrics={'train_runtime': 3601.2574, 'train_samples_per_second': 20.799, 'train_steps_per_second': 0.651, 'total_flos': 1.0156085725888512e+16, 'train_loss': 0.04599613803500453, 'epoch': 3.0})

In [15]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=fdf34b9e5dc1c8a8682a824d4d204019d8eb6ec72fd87d342af2bc489c8cbb0d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [17]:
from datasets import load_metric

bleu_metric = load_metric("bleu")
rouge_metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute BLEU and ROUGE scores
    bleu_score = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    rouge_scores = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    return {
        "bleu": bleu_score["score"],
        "rouge2": rouge_scores["rouge2"].mid.fmeasure,
        # Add other ROUGE scores if needed
    }


from transformers import AutoModelForSeq2SeqLM

# Load the model from the checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/results/checkpoint-2000")

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    eval_dataset=tokenized_datasets["test"],  # Use the test dataset for evaluation
    compute_metrics=compute_metrics
)

results = trainer.evaluate(tokenized_datasets["test"])
results

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


ValueError: Got a string but expected a list instead: 'Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die Die D